In [4]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import optuna
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from tensorflow.keras.layers import ConvLSTM1D, Flatten, Dense


class WeightLogger(tf.keras.callbacks.Callback):
    def __init__(self, layer_index=0):
        self.layer_index = layer_index
        self.weights_per_epoch = []

    def on_epoch_end(self, epoch, logs=None):
        weights = self.model.layers[self.layer_index].get_weights()[0]
        self.weights_per_epoch.append(weights.copy())


def load_and_prepare_data(csv_path, production_column='production', window_size=24):
    df = pd.read_csv(csv_path, sep=',')
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(df.values)
    target_scaler = MinMaxScaler()
    target_scaler.fit(df[[production_column]])
    target_col_idx = df.columns.get_loc(production_column)
    x, y = [], []
    for i in range(window_size, len(data_scaled)):
        x.append(data_scaled[i-window_size:i])
        y.append(data_scaled[i, target_col_idx])
    x, y = np.array(x), np.array(y)
    train_split_index = int(0.8 * len(x))
    test_split_index = int(0.9 * len(x))
    x_train, y_train = x[:train_split_index], y[:train_split_index]
    x_test, y_test = x[train_split_index:test_split_index], y[train_split_index:test_split_index]
    x_val, y_val = x[test_split_index:], y[test_split_index:]
    x_train_conv = np.expand_dims(x_train, axis=2)
    x_test_conv = np.expand_dims(x_test, axis=2)
    x_val_conv = np.expand_dims(x_val, axis=2)
    return x_train_conv, y_train, x_test_conv, y_test, x_val_conv, y_val, df, target_scaler


def build_convlstm_model(lr, filters1, filters2, dense_units, input_shape):
    model = Sequential([
        ConvLSTM1D(filters=int(filters1), kernel_size=(1,), activation='tanh',
                   return_sequences=True, input_shape=input_shape),
        ConvLSTM1D(filters=int(filters2), kernel_size=(1,), activation='tanh', return_sequences=False),
        Flatten(),
        Dense(units=int(dense_units), activation='relu'),
        Dense(1, activation="linear")
    ])
    optimizer = Adam(learning_rate=lr)
    model.compile(loss="mae", optimizer=optimizer)
    return model


def train_and_evaluate_model(model, x_train, y_train, x_val, y_val,
                             epochs=50, batch_size=512, verbose=0, dataset_name="dataset"):
    stop_early = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    weight_logger = WeightLogger(layer_index=0)
    start_time = time.time()
    history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=verbose,
                        callbacks=[stop_early, weight_logger])
    training_time = time.time() - start_time
    w00 = [w[0, 0] for w in weight_logger.weights_per_epoch]
    plt.figure(figsize=(8, 4))
    plt.plot(w00)
    plt.xlabel("Époque")
    plt.ylabel("Poids [0,0]")
    plt.title("Évolution du poids [0,0]")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_poids_w00.png")
    plt.close()
    plt.figure(figsize=(10, 4))
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title("Loss par époque")
    plt.xlabel("Époque")
    plt.ylabel("MAE")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{dataset_name}_loss_curve.png")
    plt.close()
    return history, training_time, weight_logger


def inference_and_plot(model, x_test, y_test, target_scaler, dataset_name): 
    preds = model.predict(x_test)
    y_test_real = target_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_real = target_scaler.inverse_transform(preds.reshape(-1, 1)).flatten()
    mae = mean_absolute_error(y_test_real, y_pred_real)
    mse = mean_squared_error(y_test_real, y_pred_real)
    r2 = r2_score(y_test_real, y_pred_real)
    plt.figure(figsize=(10, 6))
    plt.plot(y_test_real, label="Vrai")
    plt.plot(y_pred_real, label="Prévu")
    plt.legend()
    plt.title(f"{dataset_name} - MAE: {mae:.4f} | R²: {r2:.4f} | MSE: {mse:.4f}")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_courbe_perf.png")
    plt.close()
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test_real, y_pred_real, alpha=0.7, color='orange')
    plt.plot([min(y_test_real), max(y_test_real)], [min(y_test_real), max(y_test_real)], 'r--')
    plt.xlabel("Réel")
    plt.ylabel("Prédit")
    plt.title("Scatter plot")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_scatter_perf.png")
    plt.close()
    errors = y_test_real - y_pred_real
    plt.figure(figsize=(8, 4))
    plt.hist(errors, bins=30, color='orange', edgecolor='black')
    plt.title("Histogramme des erreurs")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_hist_errors.png")
    plt.close()
    # Corriger division par zéro pour les erreurs en pourcentage
    safe_y_test_real = np.where(y_test_real == 0, np.nan, y_test_real)
    
    df_stats = pd.DataFrame({
        "Y_test": y_test_real,
        "Y_pred": y_pred_real,
        "Error": errors,
        "Error_Percent": np.abs(errors) / safe_y_test_real * 100
    })
    
    # Sauvegarder les statistiques sans générer d'avertissements
    df_stats.describe().to_csv(f"{dataset_name}_stats_erreurs.csv")
    return mae, mse, r2


def objective(trial, input_shape, x_train, y_train, x_val, y_val):
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    f1 = trial.suggest_int('filters1', 32, 128)
    f2 = trial.suggest_int('filters2', 32, 128)
    dense = trial.suggest_int('dense_units', 32, 128)
    model = build_convlstm_model(lr, f1, f2, dense, input_shape)
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
                        epochs=10, batch_size=256, verbose=0)
    return min(history.history['val_loss'])


def run_experiment(csv_path, dataset_name):
    x_train, y_train, x_test, y_test, x_val, y_val, df, target_scaler = load_and_prepare_data(csv_path)
    input_shape = x_train.shape[1:]
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, input_shape, x_train, y_train, x_val, y_val), n_trials=100)
    best = study.best_params
    model = build_convlstm_model(best['lr'], best['filters1'], best['filters2'], best['dense_units'], input_shape)
    history, training_time, _ = train_and_evaluate_model(model, x_train, y_train, x_val, y_val,
                                                         epochs=700, dataset_name=dataset_name)
    mae, mse, r2 = inference_and_plot(model, x_test, y_test, target_scaler, dataset_name)
    print(f"{dataset_name} — MAE: {mae:.4f}, MSE: {mse:.4f}, R²: {r2:.4f}")


if __name__ == "__main__":
    run_experiment("scaled_dataset.csv", "scaled_dataset")


[I 2025-06-13 14:44:19,394] A new study created in memory with name: no-name-9d503ad0-f11e-40ca-9668-42fcf018039e
C:\Users\mymar\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[I 2025-06-13 14:45:08,086] Trial 0 finished with value: 0.025153128430247307 and parameters: {'lr': 0.00029517351520142707, 'filters1': 74, 'filters2': 45, 'dense_units': 118}. Best is trial 0 with value: 0.025153128430247307.


34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step
scaled_dataset — MAE: 11.5322, MSE: 227.9534, R²: 0.2941


In [2]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import optuna
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from tensorflow.keras.layers import ConvLSTM1D, Flatten, Dense


class WeightLogger(tf.keras.callbacks.Callback):
    def __init__(self, layer_index=0):
        self.layer_index = layer_index
        self.weights_per_epoch = []

    def on_epoch_end(self, epoch, logs=None):
        weights = self.model.layers[self.layer_index].get_weights()[0]
        self.weights_per_epoch.append(weights.copy())


def load_and_prepare_data(csv_path, production_column='production', window_size=24):
    # === 1. Chargement des données ===
    df = pd.read_csv("full_dataset.csv",sep=";")  # Remplace par ton CSV réel
    # 1.1 Conversion de la colonne date
    df['date1'] = pd.to_datetime(df['date'], dayfirst=True)
    
    # 1.2 Création des features temporelles cycliques
    df['dayofweek'] = df['date1'].dt.dayofweek
    df['month'] = df['date1'].dt.month
    
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    
    df.drop(columns=['month', 'dayofweek'], inplace=True)
    
    # 1.3 Supprimer les colonnes non numériques avant normalisation
    df = df.select_dtypes(include=[np.number])
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(df.values)
    target_scaler = MinMaxScaler()
    target_scaler.fit(df[[production_column]])
    target_col_idx = df.columns.get_loc(production_column)
    x, y = [], []
    for i in range(window_size, len(data_scaled)):
        x.append(data_scaled[i-window_size:i])
        y.append(data_scaled[i, target_col_idx])
    x, y = np.array(x), np.array(y)
    train_split_index = int(0.8 * len(x))
    test_split_index = int(0.9 * len(x))
    x_train, y_train = x[:train_split_index], y[:train_split_index]
    x_test, y_test = x[train_split_index:test_split_index], y[train_split_index:test_split_index]
    x_val, y_val = x[test_split_index:], y[test_split_index:]
    x_train_conv = np.expand_dims(x_train, axis=2)
    x_test_conv = np.expand_dims(x_test, axis=2)
    x_val_conv = np.expand_dims(x_val, axis=2)
    return x_train_conv, y_train, x_test_conv, y_test, x_val_conv, y_val, df, target_scaler


def build_convlstm_model(lr, filters1, filters2, dense_units, input_shape):
    model = Sequential([
        ConvLSTM1D(filters=int(filters1), kernel_size=(1,), activation='tanh',
                   return_sequences=True, input_shape=input_shape),
        ConvLSTM1D(filters=int(filters2), kernel_size=(1,), activation='tanh', return_sequences=False),
        Flatten(),
        Dense(units=int(dense_units), activation='relu'),
        Dense(1, activation="linear")
    ])
    optimizer = Adam(learning_rate=lr)
    model.compile(loss="mae", optimizer=optimizer)
    return model


def train_and_evaluate_model(model, x_train, y_train, x_val, y_val,
                             epochs=50, batch_size=512, verbose=0, dataset_name="dataset"):
    stop_early = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    weight_logger = WeightLogger(layer_index=0)
    start_time = time.time()
    history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=verbose,
                        callbacks=[stop_early, weight_logger])
    training_time = time.time() - start_time
    w00 = [w[0, 0] for w in weight_logger.weights_per_epoch]
    plt.figure(figsize=(8, 4))
    plt.plot(w00)
    plt.xlabel("Époque")
    plt.ylabel("Poids [0,0]")
    plt.title("Évolution du poids [0,0]")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_poids_w00.png")
    plt.close()
    plt.figure(figsize=(10, 4))
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title("Loss par époque")
    plt.xlabel("Époque")
    plt.ylabel("MAE")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{dataset_name}_loss_curve.png")
    plt.close()
    return history, training_time, weight_logger


def inference_and_plot(model, x_test, y_test, target_scaler, dataset_name):
    preds = model.predict(x_test)
    y_test_real = target_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_real = target_scaler.inverse_transform(preds.reshape(-1, 1)).flatten()
    mae = mean_absolute_error(y_test_real, y_pred_real)
    mse = mean_squared_error(y_test_real, y_pred_real)
    r2 = r2_score(y_test_real, y_pred_real)
    plt.figure(figsize=(10, 6))
    plt.plot(y_test_real, label="Vrai")
    plt.plot(y_pred_real, label="Prévu")
    plt.legend()
    plt.title(f"{dataset_name} - MAE: {mae:.4f} | R²: {r2:.4f} | MSE: {mse:.4f}")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_courbe_perf.png")
    plt.close()
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test_real, y_pred_real, alpha=0.7, color='orange')
    plt.plot([min(y_test_real), max(y_test_real)], [min(y_test_real), max(y_test_real)], 'r--')
    plt.xlabel("Réel")
    plt.ylabel("Prédit")
    plt.title("Scatter plot")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_scatter_perf.png")
    plt.close()
    errors = y_test_real - y_pred_real
    plt.figure(figsize=(8, 4))
    plt.hist(errors, bins=30, color='orange', edgecolor='black')
    plt.title("Histogramme des erreurs")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_hist_errors.png")
    plt.close()
    # Éviter la division par zéro pour les erreurs en pourcentage
    safe_y_test_real = np.where(y_test_real == 0, np.nan, y_test_real)
    
    df_stats = pd.DataFrame({
        "Y_test": y_test_real,
        "Y_pred": y_pred_real,
        "Error": errors,
        "Error_Percent": np.abs(errors) / safe_y_test_real * 100
    })
    
    # Sauvegarde sans warning
    df_stats.describe().to_csv(f"{dataset_name}_stats_erreurs.csv")
    return mae, mse, r2


def objective(trial, input_shape, x_train, y_train, x_val, y_val):
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    f1 = trial.suggest_int('filters1', 32, 128)
    f2 = trial.suggest_int('filters2', 32, 128)
    dense = trial.suggest_int('dense_units', 32, 128)
    model = build_convlstm_model(lr, f1, f2, dense, input_shape)
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
                        epochs=10, batch_size=256, verbose=0)
    return min(history.history['val_loss'])


def run_experiment(csv_path, dataset_name):
    x_train, y_train, x_test, y_test, x_val, y_val, df, target_scaler = load_and_prepare_data(csv_path)
    input_shape = x_train.shape[1:]
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, input_shape, x_train, y_train, x_val, y_val), n_trials=100)
    best = study.best_params
    model = build_convlstm_model(best['lr'], best['filters1'], best['filters2'], best['dense_units'], input_shape)
    history, training_time, _ = train_and_evaluate_model(model, x_train, y_train, x_val, y_val,
                                                         epochs=700, dataset_name=dataset_name)
    mae, mse, r2 = inference_and_plot(model, x_test, y_test, target_scaler, dataset_name)
    print(f"{dataset_name} — MAE: {mae:.4f}, MSE: {mse:.4f}, R²: {r2:.4f}")


if __name__ == "__main__":
    run_experiment("full_dataset.csv", "full_dataset")


[I 2025-06-13 14:35:55,368] A new study created in memory with name: no-name-d3712ccc-5fa6-4ba4-ba5c-2f6a9f0e2b79
C:\Users\mymar\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[I 2025-06-13 14:37:05,543] Trial 0 finished with value: 0.02478390745818615 and parameters: {'lr': 0.0026302982000410034, 'filters1': 69, 'filters2': 98, 'dense_units': 96}. Best is trial 0 with value: 0.02478390745818615.


34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step
full_dataset — MAE: 11.7640, MSE: 235.1456, R²: 0.2789
